In [1]:
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient
from collections import Counter
%matplotlib inline

Get freelancer profile by key

Endpoint
GET /api/profiles/v1/providers/{profile_key}.{format}

In [2]:
def json_prep(in_file):
    # read the entire file `ainto a python array
    with open(in_file, 'rb') as f:
        data = f.readlines()

    # remove the trailing "\n" from each line
    data = map(lambda x: x.rstrip(), data)

    # each element of 'data' is an individual JSON object.
    # i want to convert it into an *array* of JSON objects
    # which, in and of itself, is one large JSON object
    # basically... add square brackets to the beginning
    # and end, and have all the individual business JSON objects
    # separated by a comma
    data_json_str = "[" + ','.join(data) + "]"

    data_list_of_dicts = json.loads(data_json_str)
    
    out_df = pd.read_json(data_json_str)
    return out_df

## Make lists of dicts out of JSON data

In [80]:
# Get IDs that were scraped
infile = '../../data/profiles_by_skill/ids_da.txt'

ids_list = list()
with open(infile) as f:
    for line in f:
        ids_list.append(line.strip())

In [81]:
print len(ids_list)
ids_set = set(ids_list)
print len(ids_set)

209788
209788


In [83]:
print len(ids_list[:128175])
ids_set = set(ids_list[:128175])
print len(ids_set)

128175
128175


In [5]:
jobs_df = json_prep('../../data/jobs_da_0.txt')

In [11]:
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7882 entries, 0 to 7881
Data columns (total 38 columns):
amount                         7882 non-null object
assignment_info                7882 non-null object
assignments                    7882 non-null object
buyer                          7882 non-null object
candidates                     7882 non-null object
ciphertext                     7882 non-null object
engagement_weeks               7882 non-null object
interviewees_total_active      7882 non-null int64
job_category_level_one         7882 non-null object
job_category_level_two         7882 non-null object
job_type                       7882 non-null object
op_additional_questions        7882 non-null object
op_attached_doc                7882 non-null object
op_cny_upm_verified            7882 non-null int64
op_contractor_tier             7882 non-null object
op_ctime                       7882 non-null int64
op_description                 7882 non-null object
op_engagemen

In [8]:
hourly_jobs_df = jobs_df[jobs_df['job_type']=='Hourly']

In [9]:
hourly_jobs_df.T.head(50)

,2,3,7,9,12,16,17,18,20,24,...,7860,7866,7867,7873,7874,7875,7878,7879,7880,7881
amount,,,,,,,,,,,...,,,0.00,0.00,,,,,,
assignment_info,"{u'info': {u'total_charge': u'498.33', u'end_d...","{u'info': [{u'total_charge': u'0', u'end_data'...","{u'info': {u'end_data': u'1318809600000', u'ci...","{u'info': {u'end_data': u'1436313600000', u'ci...","{u'info': {u'end_data': u'1432252800000', u'ci...","{u'info': {u'end_data': u'1373673600000', u'ci...","{u'info': [{u'total_charge': u'38288', u'end_d...","{u'info': [{u'end_data': u'1379548800000', u'c...","{u'info': {u'total_charge': u'1111.01', u'end_...","{u'info': {u'end_data': u'1281398400000', u'ci...",...,"{u'info': {u'total_charge': u'372.66', u'end_d...","{u'info': {u'end_data': u'1380585600000', u'ci...","{u'info': {u'total_charge': u'212', u'end_data...","{u'info': [{u'end_data': u'1451347200000', u'c...","{u'info': [{u'end_data': u'1359590400000', u'c...","{u'info': {u'end_data': u'1349740800000', u'ci...","{u'info': {u'end_data': u'1354233600000', u'ci...","{u'info': {u'end_data': u'1342915200000', u'ci...","{u'info': {u'end_data': u'1282089600000', u'ci...","{u'info': {u'end_data': u'1356393600000', u'ci..."
assignments,"{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_total_hours': u'3.33', ...","{u'assignment': [{u'as_to': u'12/2013', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_total_hours': u'6', u'a...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'01/2012', u'as_r...",...,"{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_total_hours': u'0', u'a...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_total_hours': u'52', u'...","{u'assignment': [{u'as_to': u'02/2016', u'as_r...","{u'assignment': {u'as_to': u'11/2012', u'as_ra...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'01/2011', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r..."
buyer,{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,"{u'op_timezone': u'UTC+10:00 Brisbane', u'op_t...",{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,"{u'op_timezone': u'UTC+07:00 Bangkok, Jakarta,...","{u'op_timezone': u'UTC+01:00 Prague, Warsaw', ...","{u'op_timezone': u'UTC+01:00 Prague, Warsaw', ...",...,{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,"{u'op_timezone': u'UTC+12:00 Wellington, Auckl...","{u'op_timezone': u'UTC+01:00 Berlin, Stockholm...","{u'op_timezone': u'UTC+05:00 Islamabad, Karach...",{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,"{u'op_timezone': u'UTC+01:00 Berlin, Stockholm...",{u'op_timezone': u'UTC (Coordinated Universal ...,"{u'op_timezone': u'UTC+08:00 Hong Kong SAR, Pe...",{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,"{u'op_timezone': u'UTC+01:00 Berlin, Stockholm..."
candidates,{u'candidate': [{u'ciphertext': u'~01f1c24717c...,{u'candidate': [{u'ciphertext': u'~01cde4ef409...,{u'candidate': [{u'ciphertext': u'~010973123f7...,{u'candidate': [{u'ciphertext': u'~011a4fcf20c...,{u'candidate': [{u'ciphertext': u'~01ef5d7d638...,{u'candidate': [{u'ciphertext': u'~0193bea4825...,{u'candidate': [{u'ciphertext': u'~010e62428f0...,{u'candidate': [{u'ciphertext': u'~015538cf30e...,{u'candidate': [{u'ciphertext': u'~01982a3377a...,{u'candidate': [{u'ciphertext': u'~011daef8015...,...,{u'candidate': [{u'ciphertext': u'~0161d730c5e...,{u'candidate': [{u'ciphertext': u'~017bdefaaaa...,{u'candidate': [{u'ciphertext': u'~01a5e31b786...,{u'candidate': {u'ciphertext': u'~01e495c52e01...,{u'candidate': [{u'

In [90]:
hourly_jobs_df.head(2)

,amount,assignment_info,assignments,buyer,candidates,ciphertext,engagement_weeks,interviewees_total_active,job_category_level_one,job_category_level_two,...,op_pref_has_portfolio,op_pref_hourly_rate_max,op_pref_hourly_rate_min,op_pref_location,op_pref_odesk_hours,op_required_skills,op_title,op_tot_cand,op_tot_feedback,ui_opening_status
2,,"{u'info': {u'total_charge': u'498.33', u'end_d...","{u'assignment': [{u'as_to': u'Present', u'as_r...",{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,{u'candidate': [{u'ciphertext': u'~01f1c24717c...,~0103703f207be9f938,Less than 1 week,0,,,...,0,0,0,,0,{u'op_required_skill': {u'skill': u'wordpress'}},WordPress Templates,62,9,Filled
3,,"{u'info': [{u'total_charge': u'0', u'end_data'...","{u'assignment': [{u'as_total_hours': u'3.33', ...",{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,{u'candidate': [{u'ciphertext': u'~01cde4ef409...,~01037164e7e1613e2f,More than 6 months,0,,,...,0,,,,0,{u'op_required_skill': [{u'skill': u'internet-...,Generate lead for special niche,14,23,Filled


In [ ]:
def triage(dict_, level=0):
    if not(isinstance(dict_, float) or isinstance(dict_, basestring)):
        print type(dict_)
        for key, val in dict_.iteritems():
            print ' '*level*2, key
            if isinstance(val, dict):
                triage(val, level=level+1)
            elif isinstance(val, list):
                for index, item in enumerate(val): 
                    print ' '*level*2, 'item', index
                    triage(item, level=level+1)
            elif isinstance(val, basestring):
                print ' '*(level+1)*2, '=', val

In [53]:
candidate_list = list()
for index, candidates in enumerate(jobs_df['candidates']):
    if isinstance(candidates, dict):
        if isinstance(candidates['candidate'], dict):
            # Is the only candidate
            candidate_list.append(job_seekers['candidate']['ciphertext'])
        elif isinstance(candidates['candidate'], list):
            # Is a list of candidates
            for candidate in candidates['candidate']:
                candidate_list.append(candidate['ciphertext'])
    else:
        print index


52
80
83
90
107
117
142
143
155
160
182
183
203
224
225
273
279
280
287
331
356
370
374
402
420
438
455
462
464
496
515
655
676
725
741
758
780
800
808
824
826
843
847
865
878
898
928
933
989
1030
1033
1083
1104
1118
1127
1149
1170
1198
1200
1244
1248
1250
1259
1282
1318
1344
1348
1351
1378
1507
1543
1552
1589
1600
1602
1640
1657
1705
1731
1737
1756
1758
1772
1801
1822
1826
1828
1838
1858
1884
1886
1896
1924
1948
1951
1982
2021
2028
2057
2080
2086
2091
2103
2107
2136
2179
2213
2218
2241
2245
2280
2283
2297
2333
2336
2356
2374
2441
2442
2476
2494
2506
2513
2548
2573
2605
2607
2623
2626
2637
2665
2669
2670
2682
2692
2710
2732
2740
2760
2803
2830
2831
2875
2898
2904
2931
2946
2958
3005
3008
3077
3085
3096
3104
3134
3138
3144
3153
3174
3213
3224
3244
3252
3261
3277
3319
3334
3335
3346
3353
3382
3393
3412
3415
3425
3434
3439
3446
3454
3471
3472
3480
3516
3525
3530
3531
3534
3543
3554
3564
3570
3583
3586
3589
3667
3674
3676
3742
3773
3788
3790
3818
3819
3847
3875
3887
3939
3945
3954
3976
400

In [91]:
for x in range(0,10):
    print x
    job_cat = jobs_df.op_job_category_v2[x]
    triage(job_cat)

0
<type 'dict'>
 op_job_category_v
<type 'dict'>
   name
     = Web & Mobile Design
   groups
<type 'dict'>
     group
<type 'dict'>
       uid
         = 531770282580668418
       name
         = Web, Mobile & Software Dev
1
<type 'dict'>
 op_job_category_v
<type 'dict'>
   name
     = Article & Blog Writing
   groups
<type 'dict'>
     group
<type 'dict'>
       uid
         = 531770282580668423
       name
         = Writing
2
<type 'dict'>
 op_job_category_v
<type 'dict'>
   name
     = Web Development
   groups
<type 'dict'>
     group
<type 'dict'>
       uid
         = 531770282580668418
       name
         = Web, Mobile & Software Dev
3
<type 'dict'>
 op_job_category_v
<type 'dict'>
   name
     = Lead Generation
   groups
<type 'dict'>
     group
<type 'dict'>
       name
         = Sales & Marketing
       uid
         = 531770282580668422
4
<type 'dict'>
 op_job_category_v
<type 'dict'>
   name
     = Other - Software Development
   groups
<type 'dict'>
     group
<type 'di

In [112]:
job_cats = list()
for index, cat2_info in enumerate(jobs_df['op_job_category_v2']):
    cat_found = False
    if isinstance(cat2_info, dict):
        cat1_info = cat2_info['op_job_category_v']
        if isinstance(cat1_info, dict):
            groups = cat1_info['groups']
            if isinstance(groups, dict):
                group = groups['group']
                if 'name' in group:
                    job_cats.append(group['name'])
                    cat_found = True
    if cat_found == False:
        job_cats.append('')


In [113]:
print len(job_cats)

7882


In [114]:
job_cats_array = np.array(job_cats)

In [115]:
print len(job_cats_array)

7882


In [116]:
len(job_cats_array == u'Data Science & Analytics')

7882

In [117]:
len(jobs_df)

7882

In [118]:
jobs_da = jobs_df[job_cats_array == u'Data Science & Analytics']

In [119]:
len(jobs_da)

177

In [95]:
print Counter(job_cats)

Counter({u'Web, Mobile & Software Dev': 5776, u'Admin Support': 491, u'Writing': 335, u'IT & Networking': 323, u'Design & Creative': 281, u'Sales & Marketing': 243, u'Data Science & Analytics': 177, u'Translation': 98, u'Accounting & Consulting': 90, u'Engineering & Architecture': 40, u'Customer Service': 20, u'Legal': 2})


In [84]:
got_job = list()
for index, assignment_info in enumerate(jobs_df['assignment_info']):
    if isinstance(assignment_info, dict):
        info = assignment_info['info']
        if isinstance(info, dict):
            got_job.append(info['ciphertext_developer_recno'])
        elif isinstance(info, list):
            for job_getter in info:
                got_job.append(job_getter['ciphertext_developer_recno'])
        else:
            print index
#             pass
    else:
        print index
#         pass

In [86]:
print len(got_job)
got_job_set = set(got_job)
print len(got_job_set)

11382
8261


In [89]:
new_ids = got_job_set.difference(ids_set)
print len(new_ids)

3284


In [10]:
hourly_jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3785 entries, 2 to 7881
Data columns (total 38 columns):
amount                         3785 non-null object
assignment_info                3785 non-null object
assignments                    3785 non-null object
buyer                          3785 non-null object
candidates                     3785 non-null object
ciphertext                     3785 non-null object
engagement_weeks               3785 non-null object
interviewees_total_active      3785 non-null int64
job_category_level_one         3785 non-null object
job_category_level_two         3785 non-null object
job_type                       3785 non-null object
op_additional_questions        3785 non-null object
op_attached_doc                3785 non-null object
op_cny_upm_verified            3785 non-null int64
op_contractor_tier             3785 non-null object
op_ctime                       3785 non-null int64
op_description                 3785 non-null object
op_engagemen

In [13]:


for x in range(1000,1020):
    print x
    jobs = jobs_df.candidates[x]
    triage(jobs)

1000
<type 'dict'>
 candidate
<type 'dict'>
   ciphertext
     = ~016a17056c22b76516
   create_date_ts
     = 1321293465
1001
<type 'dict'>
 candidate
 item 0
<type 'dict'>
   ciphertext
     = ~0145de61596309fade
   create_date_ts
     = 1320176193
 item 1
<type 'dict'>
   ciphertext
     = ~0128b526406e355bb6
   create_date_ts
     = 1320171285
 item 2
<type 'dict'>
   ciphertext
     = ~013efb3c61532fcb5b
   create_date_ts
     = 1320170811
 item 3
<type 'dict'>
   ciphertext
     = ~0188c7f1e91a160ce3
   create_date_ts
     = 1320169991
 item 4
<type 'dict'>
   ciphertext
     = ~0182a346cfa0b27a13
   create_date_ts
     = 1320167614
 item 5
<type 'dict'>
   ciphertext
     = ~01d563e486adfc7365
   create_date_ts
     = 1320166856
 item 6
<type 'dict'>
   ciphertext
     = ~0193c3815be0471e98
   create_date_ts
     = 1320166094
 item 7
<type 'dict'>
   ciphertext
     = ~01c18ff1af435e9e8b
   create_date_ts
     = 1320165812
 item 8
<type 'dict'>
   ciphertext
     = ~01ccbadcfcad6

In [ ]:
for x in range(20):
    print x
    jobs = df.assignments[x]
    for k1, v1 in jobs.iteritems():        
        print 'k1 = ', k1
        if isinstance(v1, basestring):
            print 'basestring'
            pass
        elif isinstance(v1, dict):
            print 'dict'
            for k2, v2 in v1.iteritems():
                print '  k2 = ', k2
                if isinstance(v2, basestring):
                    print '  basestring'
                    pass
                elif isinstance(v2, dict):
                    print '  dict'
                    for k3, v3 in v2.iteritems():
                        print '    k3 = ' + k3
                elif isinstance(v2, list):
                    print '  list'
                    for index2, item in enumerate(v2):
                        print '    item', index2
                        if isinstance(item, dict):
                            for k4, v4 in item.iteritems():
                                print '    k4 = ', k4
        elif isinstance(v1, list):
            print 'list'
            for item in v1:
                print item
    print
    

In [ ]:
def triage(dict_, level=0):
    if not(isinstance(dict_, float) or isinstance(dict_, basestring)):
        print type(dict_)
        for key, val in dict_.iteritems():
            print ' '*level*2, key
            if isinstance(val, dict):
                triage(val, level=level+1)
            elif isinstance(val, list):
                for index, item in enumerate(val): 
                    print ' '*level*2, 'item', index
                    triage(item, level=level+1)
            elif isinstance(val, basestring):
                print ' '*(level+1)*2, '=', val

for x in range(1000,1020):
    print x
    jobs = df.dev_job_categories_v2[x]
    triage(jobs)